In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kukuroo3/body-signal-of-smoking")

print("Path to dataset files:", path)

C:\Users\Tom\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Tom\.cache\kagglehub\datasets\kukuroo3\body-signal-of-smoking\versions\2


In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(path+"/smoking.csv")
df = df.drop(columns = ["oral", "tartar", "ID"])
df["gender"] = df["gender"].map({"F": -1, "M": 1})

y_total = df.pop("smoking").astype(int)
X_total = df

X, X_check, y, y_check = train_test_split(
    X_total,
    y_total,
    test_size = 5000,
    stratify = y_total,
    random_state = 123
)


y
38519    1
15530    0
10828    0
39365    1
17799    1
        ..
33781    0
37688    0
1305     0
26341    1
21478    0
Name: smoking, Length: 50692, dtype: int64
y_check
32193    0
49703    1
30883    0
6336     0
26100    1
        ..
18598    0
6924     0
18409    1
16275    1
44337    1
Name: smoking, Length: 5000, dtype: int64


In [68]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report

rf_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("rf", RandomForestClassifier(
        n_estimators=4000,
        min_samples_leaf=2,
        class_weight="balanced_subsample",
        n_jobs=-1,
        random_state=123
    ))
])

rf_pipe.fit(X,y)

proba = rf_pipe.predict_proba(X_check)[:, 1]
pred  = (proba >= 0.5).astype(int)

print(classification_report(y_check, pred, digits=3))

              precision    recall  f1-score   support

           0      0.898     0.851     0.874      3164
           1      0.764     0.833     0.797      1836

    accuracy                          0.844      5000
   macro avg      0.831     0.842     0.835      5000
weighted avg      0.849     0.844     0.845      5000



In [71]:

rf = rf_pipe.named_steps["rf"]
import pandas as pd
imp = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nTop 10 features:\n", imp.head(10))


Top 10 features:
 gender          0.152799
hemoglobin      0.094119
Gtp             0.083308
height(cm)      0.080686
triglyceride    0.060326
waist(cm)       0.044864
LDL             0.042866
ALT             0.042277
Cholesterol     0.042148
HDL             0.042036
dtype: float64


In [ ]:
# upsampling auf Female / Female + smokers

# RandomForestClassifier, Support Vector Machine, MLP (Multi Layer)

# Hyperparameter, Gridsearch

# Correlation, Signifikanztests

# Literaturrecherche (verweis auf die Correlations, etc.)

#